In [9]:
# Import dependencies and API key

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import requests

from datetime import datetime
from citipy import citipy

from config import weather_api_key

In [10]:
# Create a set of random latitude and longitude combinations

lats = np.random.uniform(low=-90, high=90, size=1500)
lngs = np.random.uniform(low=-180, high=180, size=1500)

lat_lngs = zip(lats, lngs)

lat_lngs

In [11]:
# Add the latitudes and longitudes to a list

coordinates = list(lat_lngs)

In [12]:
# Create a list to hold the cities

cities = []

# Find the nearest city for each set of coordinates

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # Add the city to the cities list if it's not already in there
    if city not in cities:
        cities.append(city)

# Print the city count

len(cities)

636

In [14]:
# Create starting URL for API call
# Format from OpenWeather:
# https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}

url = "https://api.openweathermap.org/data/2.5/weather?units=imperial&appid=" + weather_api_key

'https://api.openweathermap.org/data/2.5/weather?units=imperial&appid=5550e2938f6e2d7dfc4df5577f452a71'

In [15]:
url_boston = url + "&q=" + "Boston"
url_boston

'https://api.openweathermap.org/data/2.5/weather?units=imperial&appid=5550e2938f6e2d7dfc4df5577f452a71&q=Boston'

In [5]:
# Create an empty list to hold the weather data

city_data = []

# Print the beginning of the log

print("Beginning data retrieval               ")
print("---------------------------------------")

# Create counters

record_count = 1
set_count = 1

Beginning data retrieval               
---------------------------------------


In [ ]:
# Loop through all our cities
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    # If i is at least 50 and is divisible by 50, then we have started a new set.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        #After looping through 50 cities, we need to pause for a minute to avoid making too many API calls/minute.
        time.sleep(60)
        
    # Create endpoint URL for a city
    # If a city name has a space, we need to replace it with a plus sign in the URl
    city_url = url + "&q=" + city.replace(" ", "+")

# Log the URl, the record and set numbers, and the city
print(f"Processing record {record_count} of set {set_count} | {city}")

# Add one to the record count
record_count += 1

# Run an API request for each city
try:
    #Retrieve data; parse JSON
    city_weather = requests.get(city_url).json()
    
    #Get the data points we need
    city_lat = city_weather["coord"]["lat"]
    city_lng = city_weather["coord"]["lon"]
    city_max_temp = city_weather["main"]["temp_max"]
    city_humidity = city_weather["main"]["humidity"]
    city_clouds = city_weather["clouds"]["all"]
    city_wind = city_weather["wind"]["speed"]
    city_country = city_weather["sys"]["country"]

    
